In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')
# データフレームを表示して確認
df.head()

In [ ]:
# アンケートの主要キーワードをLLMで抽出する関数
def extract_keywords(text):
    # 2. データをLLM用にテキスト形式に変換
    # プロンプトの作成
    prompt_text = f"以下のアンケートから主要なキーワードをカンマ区切りで抽出してください。\nアンケート: {text}"

    # 3. OpenAI APIの呼び出し
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # キーワード抽出結果を取得
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [ ]:
# テストコードで確認
text = extract_keywords("pandasは、Excelファイル（`.xls`、`.xlsx` 形式）の読み書きに加え、データの集計や変換、フィルタリングといった複雑な操作を手軽に行えるライブラリです。")
print(text)

In [ ]:
# 4. 分析結果をデータフレームに変換

# B列（キーワード）を初期化
df['キーワード'] = ""

# for文で各アンケート行に対してキーワード抽出を実行
for index, row in df.iterrows():
    # A列のアンケート内容を取得
    survey_text = row['アンケート']
    
    # キーワード抽出関数を呼び出し、結果をB列に保存
    keywords = extract_keywords(survey_text)
    df.at[index, 'キーワード'] = keywords

# データフレームを表示して確認
df.head()


In [ ]:
# 5. 結果をExcelファイルに保存
df.to_excel("アンケート分析結果.xlsx", index=False)


In [ ]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

# A列の各行のアンケートに対してキーワード抽出を実行し、B列に書き込む
df['キーワード'] = df['アンケート'].apply(extract_keywords)

# 結果をExcelファイルに保存
df.to_excel("アンケート分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")
実行結果：
処理を開始します。
Excelファイルに分析結果を保存しました。
